In [1]:
import hexathello.aiPlayers as aiPlayers
import hexathello.AutoPlayer as autoPlayer
import hexathello.Engine as engine
import hexathello.history as history
import hexathello.jable as jable
import hexathello.printing as printing

import numpy as np

from os import path
import os
import math
import tensorflow as tf

from os import listdir
from os.path import join, isfile


# -- Settings
game_size: int = 5
player_count: int = 2

In [2]:
matchup_dir = "data/history/matchups_1"
matchup_files = [f for f in listdir(matchup_dir) if f.endswith(".json")]
print(matchup_files)

agent_ids_v2 = [
    "1x720_total720_size5_players2_v2",
    "2x360_total720_size5_players2_v2",
    "3x240_total720_size5_players2_v2",
    "4x180_total720_size5_players2_v2",
    "5x144_total720_size5_players2_v2",
    "6x120_total720_size5_players2_v2",
    "7x102_total714_size5_players2_v2"
]

['all_matchups.json']


In [3]:

for ai_id_v2 in agent_ids_v2:
    print(f"\n Retraining agent: {ai_id_v2}")

    training_pov = []

    relevant_files = [f for f in matchup_files if ai_id_v2 in f]

    match_frames = []
    for fname in matchup_files:
        full_path = join(matchup_dir, fname)
        
        history_fromDisk: jable.JyFrame = jable.read_file(
            full_path
        )
    
        history_decoded: jable.JyFrame = history.history_fromInt(
            history_fromDisk
        )
    
        if len(history_decoded) < 20:
            print('skipping')
            continue
    
    
        povHistory: jable.JyFrame = history.povHistory_from_literalHistory(
            history_decoded
        )

        del history_decoded
        del history_fromDisk

        match_frames.append(povHistory)


        input_size: int = len(povHistory[0, 'board_state'])
        output_size: int = len(povHistory[0, 'player_action'])

    keras_path_v2 = join("data", "ai", "vary_dimensions", f"{ai_id_v2}.keras")
    if not path.isfile(keras_path_v2):
        print(f"Model not found at {keras_path_v2}")
        continue

    brain_model = tf.keras.models.load_model(keras_path_v2)
    ai_id_final = ai_id_v2.replace("v2", "final")
    keras_path_final = join("data", "ai", "vary_dimensions", f"{ai_id_final}.keras")
    os.makedirs(os.path.dirname(keras_path_final), exist_ok=True)

    # Init updated agent
    ai_agent = aiPlayers.KerasHexAgent(
        size=game_size,
        player_count=player_count,
        brain=brain_model,
        player_id=None,
        ai_id=ai_id_final
    )

    # Save only the improved model under the final name
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=keras_path_final,
        monitor='loss',
        mode='min',
        save_best_only=True
    )

    training_pov_frame: jable.JyFrame = jable.JyFrame(training_pov)

    # Train with only relevant matches
    for povHistory in match_frames:
        ai_agent.train(
            game_history=povHistory,
            epochs=10,
            callbacks=[checkpoint_cb]
        )

    print(f"Retrained and saved model as {keras_path_final}")



 Retraining agent: 1x720_total720_size5_players2_v2
Epoch 1/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.5126
Epoch 2/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.8735
Epoch 3/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.6794
Epoch 4/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.5529
Epoch 5/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4181
Epoch 6/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3484
Epoch 7/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2722
Epoch 8/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2146
Epoch 9/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1773
Epoch 10/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1593
Retrained and saved model as data\ai\vary_dimensions\1x720_total720_size5_players2_final.keras

 Retraining agent: 2x360_total720_size5_players2_v2
Epoch 1/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.7005
Epoch 2/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1